# ４章　レポーティングする仕組みを構築する10本ノック

### ノック３１：特定店舗の売上をExcelにして出力してみよう

In [ ]:
import pandas as pd
import glob
import os

m_store = pd.read_csv("m_store.csv")
m_area = pd.read_csv("m_area.csv")

current_dir = os.getcwd()
tbl_order_file = os.path.join(current_dir, "tbl_order_*.csv")
tbl_order_files = glob.glob(tbl_order_file)

order_all = pd.DataFrame()
for file in tbl_order_files:
  order_tmp = pd.read_csv(file)
  print(f"{file}:{len(order_tmp)}")
  order_all = pd.concat([order_all, order_tmp], ignore_index=True)

# 保守等店舗のデータ削除
order_all = order_all.loc[order_all["store_id"] != 999]

order_all = pd.merge(order_all, m_store, on="store_id", how="left")
order_all = pd.merge(order_all, m_area, on="area_cd", how="left")

# マスターにないコードに対応した文字列を設定
order_all.loc[order_all["takeout_flag"]==0, "takeout_name"] = "デリバリー"
order_all.loc[order_all["takeout_flag"]==1, "takeout_name"] = "お持ち帰り"

order_all.loc[order_all["status"]==0, "status_name"] = "受付"
order_all.loc[order_all["status"]==1, "status_name"] = "お支払済"
order_all.loc[order_all["status"]==2, "status_name"] = "お渡し済"
order_all.loc[order_all["status"]==9, "status_name"] = "キャンセル"

order_all.loc[:, "order_date"] = pd.to_datetime(order_all["order_accept_date"]).dt.date

order_all.head()

In [ ]:
import openpyxl

wb = openpyxl.Workbook()
ws = wb["Sheet"]
ws.cell(1,1).value = "書き込みテストです"
wb.save("test.xlsx")
wb.close()

In [ ]:
wb = openpyxl.load_workbook("test.xlsx", read_only=True)
ws = wb["Sheet"]
print(ws.cell(1,1).value)
wb.close()

In [ ]:
# テストデータの準備
store_id = 1
store_df = order_all.loc[order_all["store_id"] == store_id].copy()
store_name = store_df["store_name"].unique()[0]
store_sales_total = store_df.loc[store_df["status"].isin([1,2])]["total_amount"].sum()
store_sales_takeout = store_df.loc[store_df["status"]==1]["total_amount"].sum()
store_sales_delivery = store_df.loc[store_df["status"]==2]["total_amount"].sum()
print(f"売上額確認 {store_sales_total} = {store_sales_takeout + store_sales_delivery}")
output_df = store_df[["order_accept_date", "customer_id", "total_amount", "takeout_name", "status_name"]]
output_df.head()

In [ ]:
from openpyxl.utils.dataframe import dataframe_to_rows

store_title = f"{store_id}_{store_name}"

wb = openpyxl.Workbook()
ws = wb.active
ws.title = store_title

ws.cell(1,1).value = f"{store_title} 売り上げデータ"

# OpenPyXL ののユーティリティ dataframe_to_rows を利用
rows = dataframe_to_rows(output_df, index=False, header=True)

# 表の貼り付け位置
row_start = 3
col_start = 2

for row_no, row in enumerate(rows, row_start):
    for col_no, value in enumerate(row, col_start):
        ws.cell(row_no, col_no).value = value

filename = f"{store_title}.xlsx"
wb.save(filename)
wb.close()

### ノック３２：Excelの表を整えて出力してみよう

In [ ]:
# スタイル関係系のインポート
from openpyxl.styles import PatternFill, Border, Side, Font

openpyxl.load_workbook(filename)
ws = wb[store_title]

side = Side(style="thin", color="008080")
border = Border(top=side, bottom=side, left=side, right=side)

# データの表の部分に罫線を設定
for row in ws:
  for cell in row:
    if ws[cell.coordinate].value:
      ws[cell.coordinate].border = border

ws.cell(1,1).font = Font(bold=True, color="008080")

cell = ws.cell(3,2)
cell.fill = PatternFill(patternType="solid", fgColor="008080")
cell.value = "注文受注日時"
cell.font = Font(bold=True, color="FFFFFF")

cell = ws.cell(3,3)
cell.fill = PatternFill(patternType="solid", fgColor="008080")
cell.value = "顧客ID"
cell.font = Font(bold=True, color="FFFFFF")

cell = ws.cell(3,4)
cell.fill = PatternFill(patternType="solid", fgColor="008080")
cell.value = "購入総額"
cell.font = Font(bold=True, color="FFFFFF")

cell = ws.cell(3,5)
cell.fill = PatternFill(patternType="solid", fgColor="008080")
cell.value = "注文タイプ"
cell.font = Font(bold=True, color="FFFFFF")

cell = ws.cell(3,6)
cell.fill = PatternFill(patternType="solid", fgColor="008080")
cell.value = "注文状態"
cell.font = Font(bold=True, color="FFFFFF")

ws.column_dimensions["A"].width = 20
ws.column_dimensions["B"].width = 20
ws.column_dimensions["C"].width = 12
ws.column_dimensions["D"].width = 12
ws.column_dimensions["E"].width = 12
ws.column_dimensions["F"].width = 12

# ファイルに保存
wb.save(filename)
wb.close()


### ノック３３：売上以外のデータも出力してみよう

In [ ]:
def calc_delta(t):
    t1, t2 = t
    delta = t2 - t1
    return delta.total_seconds()/60

store_df.loc[:, "order_accept_datetime"] = pd.to_datetime(store_df["order_accept_date"])
store_df.loc[:, "delivered_datetime"] = pd.to_datetime(store_df["delivered_date"])
store_df.loc[:, "delta"] = store_df[["order_accept_datetime", "delivered_datetime"]].apply(calc_delta, axis=1)

delivery_time = store_df.groupby(["store_id"])["delta"].describe()
delivery_time

In [ ]:
openpyxl.load_workbook(filename)
ws = wb[store_title]

cell = ws.cell(1,7)
cell.value = f"配達完了までの時間"
cell.font = Font(bold=True, color="008080")

rows = dataframe_to_rows(delivery_time, index=False, header=True)

# 表の貼り付け位置
row_start = 3
col_start = 8

for row_no, row in enumerate(rows, row_start):
    for col_no, value in enumerate(row, col_start):
        cell = ws.cell(row_no, col_no)
        cell.value = value
        cell.border = border
        if row_no == row_start:
            cell.fill = PatternFill(patternType="solid", fgColor="008080")
            cell.font = Font(bold=True, color="FFFFFF")

filename = f"{store_title}.xlsx"
wb.save(filename)
wb.close()

### ノック３４：問題のある箇所を赤字で出力してみよう

In [ ]:
openpyxl.load_workbook(filename)
ws = wb[store_title]

rows = dataframe_to_rows(output_df, index=False, header=True)

# 表の貼り付け位置
row_start = 3
col_start = 2

for row_no, row in enumerate(rows, row_start):
    if row_no == row_start:
        continue
    for col_no, value in enumerate(row, col_start):
        ws.cell(row_no, col_no).value = value
        if value == "キャンセル":
            ws.cell(row_no, col_no).font = Font(bold=True, color="FF0000")

filename = f"{store_title}.xlsx"
wb.save(filename)
wb.close()

### ノック３５：エクセルのセル関数で日毎の集計をしてみよう

In [ ]:
openpyxl.load_workbook(filename)
ws = wb[store_title]

cell = ws.cell(7,7)
cell.value = "集計"
cell.font = Font(bold=True, color="008080")

cell = ws.cell(8,8)
cell.value = "データ総額"
cell.font = Font(bold=True, color="008080")

cell = ws.cell(8,10)
cell.value = f"=SUM(D4:D{ws.max_row})"

cell = ws.cell(9,8)
cell.value = " 内 決済完了額"
cell.font = Font(bold=True)

cell = ws.cell(9,10)
cell.value = f'=SUMIF(F4:F{ws.max_row},"<>"&"キャンセル", D4:D{ws.max_row})'

cell = ws.cell(10,8)
cell.value = " 内 キャンセル額"
cell.font = Font(bold=True)

cell = ws.cell(10,10)
cell.value = f'=SUMIF(F4:F{ws.max_row},"="&"キャンセル", D4:D{ws.max_row})'

filename = f"{store_title}.xlsx"
wb.save(filename)
wb.close()

### ノック３６：折れ線グラフにして出力してみよう

### ノック３７：レポートに向けてデータを準備しよう

### ノック３８：データシートに必要なデータを出力しよう

### ノック３９：サマリーシートを作成しよう

### ノック４０：店舗別にレポートをExcel出力してみよう